## **Introduction**

This notebook implements the code to handle graph traversal and constraint solving in a program path analysis.  
It contains two main components:

1. **Graph Traversal**:  
   This part uses depth-first search to traverse paths between nodes in a graph.

2. **Points-to Analysis**:  
   This part solves constraints between nodes to determine the points-to sets and propagates this information across nodes.

---

### **Graph Class**

In the `Graph` class, we traverse the graph from the `src` node to the `dst` node and store each path as a string in the `paths` set.  
The format of each path is:  
`"START->1->2->4->5->END"`, where `->` represents edges between node IDs.

---

### **CGraph Class**

In the `CGraph` class, we solve constraints iteratively. This involves:  
- Propagating points-to sets among nodes.  
- Adding copy edges.  
- Ensuring that no new copy edges are added once a fixed point is reached.

---

Now, let's proceed to define the classes and implement the algorithms. You should complete the TODO sections in the code to finalize the graph traversal and constraint solving. This will involve implementing the logic for depth-first search in the `Graph` class and completing the iterative constraint-solving process in the `CGraph` class to ensure that points-to sets are accurately propagated and no new copy edges are added after reaching a fixed point.


```

In [1]:
from collections import deque
from typing import Set, Dict, List

class Edge:
    def __init__(self, src, dst):
        assert isinstance(src, Node)
        assert isinstance(dst, Node)
        self.src = src
        self.dst = dst

    def get_src(self):
        return self.src

    def get_dst(self):
        return self.dst

class Node:
    def __init__(self, node_id: int):
        self.node_id = node_id
        self.out_edges: Set[Edge] = set()

    def get_node_id(self) -> int:
        return self.node_id

    def get_out_edges(self) -> Set['Edge']:
        return self.out_edges

    def add_out_edge(self, edge: 'Edge'):
        self.out_edges.add(edge)

class Graph:
    def __init__(self):
        self.nodes: Set[Node] = set()
        self.paths: Set[str] = set()
        self.visited: Set[Node] = set()
        self.path: List[int] = []

    def add_node(self, node: Node):
        self.nodes.add(node)

    def get_nodes(self) -> Set[Node]:
        return self.nodes

    def get_paths(self) -> Set[str]:
        return self.paths
    
    # TODO: Implement your depth-first search here to traverse each program path (once for any loop) from src to dst.
    # Add each path as a string into a set called `paths`.
    # Each path should have a format like: "START->1->2->4->5->END", where -> indicates an edge connecting two node IDs.
    def reachability(self, src: Node, dst: Node):
        # TODO: Implement your depth-first search here
        pass

class CGEdge:
    ADDR, COPY, STORE, LOAD = range(4)

    def __init__(self, src: 'CGNode', dst: 'CGNode', edge_type: int):
        self.src = src
        self.dst = dst
        self.ty = edge_type

    def get_type(self) -> int:
        return self.ty

    def get_src(self) -> 'CGNode':
        return self.src

    def get_dst(self) -> 'CGNode':
        return self.dst

class CGNode:
    def __init__(self, node_id: int):
        self.node_id = node_id
        self.points_to_set: Set[int] = set()
        self.in_edges: Set[CGEdge] = set()
        self.out_edges: Set[CGEdge] = set()

    def get_id(self) -> int:
        return self.node_id

    def get_pts(self) -> Set[int]:
        return self.points_to_set

    def get_in_edges(self) -> Set[CGEdge]:
        return self.in_edges

    def get_out_edges(self) -> Set[CGEdge]:
        return self.out_edges

    def add_in_edge(self, edge: CGEdge):
        self.in_edges.add(edge)

    def add_out_edge(self, edge: CGEdge):
        self.out_edges.add(edge)

class CGraph:
    def __init__(self):
        self.edges: Set[CGEdge] = set()
        self.id_to_node_map: Dict[int, CGNode] = {}
        self.worklist: deque[int] = deque()

    def add_node(self, node: CGNode):
        self.id_to_node_map[node.get_id()] = node

    def get_node(self, node_id: int) -> CGNode:
        return self.id_to_node_map[node_id]

    def get_pts(self, node_id: int) -> Set[int]:
        return self.get_node(node_id).get_pts()

    def add_pts(self, s: CGNode, d: CGNode) -> bool:
        return d.get_id() not in s.get_pts() and not s.get_pts().add(d.get_id())

    def union_pts(self, s: CGNode, d: CGNode) -> bool:
        changed = False
        for elem in d.get_pts():
            if elem not in s.get_pts():
                s.get_pts().add(elem)
                changed = True
        return changed

    def add_edge(self, s: CGNode, d: CGNode, edge_type: int) -> bool:
        for e in self.edges:
            if e.get_src() == s and e.get_dst() == d and e.get_type() == edge_type:
                return False
        new_edge = CGEdge(s, d, edge_type)
        s.add_out_edge(new_edge)
        d.add_in_edge(new_edge)
        self.edges.add(new_edge)
        return True

    def push_into_worklist(self, node_id: int):
        self.worklist.append(node_id)

    def pop_from_worklist(self) -> int:
        return self.worklist.popleft()
    
    # TODO: Implement constraint solving by iteratively:
    # (1) Propagating points-to sets among nodes in CGraph.
    # (2) Adding new copy edges until a fixed point is reached (i.e., no new copy edges are added).
    # The solving rules are as follows:
    # p <--ADDR-- o   =>  pts(p) = {o}
    # q <--COPY-- p   =>  pts(q) = pts(q) ∪ pts(p)
    # q <--LOAD-- p   =>  for each o ∈ pts(p): q <--COPY-- o
    # q <--STORE-- p  =>  for each o ∈ pts(q): o <--COPY-- p
    # pts(q) denotes the points-to set of node q.
    # Refer to the APIs in CGraph, including `add_pts`, `get_pts`, `union_pts`, and `add_edge` for your implementation.
    def solve_worklist(self):
        # TODO: Implement constraint solving
        pass

- **Congratulations! You've Completed the TODO Sections**:  
  Now that you've completed the TODO parts, great job! 🎉  
- Below are a few test cases to check if your code is working as expected.
- Remember, even if some tests fail, don't be discouraged—it’s a normal part of the learning process. Debugging is a valuable skill that helps you grow as a developer. Keep going, and keep improving!

---

In [2]:
def test_reachability():
    node1 = Node(1)
    node2 = Node(2)
    node3 = Node(3)
    node4 = Node(4)
    node5 = Node(5)

    edge1 = Edge(node1, node2)
    edge2 = Edge(node1, node3)
    edge3 = Edge(node2, node4)
    edge4 = Edge(node3, node4)
    edge5 = Edge(node4, node5)

    node1.add_out_edge(edge1)
    node1.add_out_edge(edge2)
    node2.add_out_edge(edge3)
    node3.add_out_edge(edge4)
    node4.add_out_edge(edge5)

    g = Graph()
    for node in [node1, node2, node3, node4, node5]:
        g.add_node(node)

    g.reachability(node1, node5)

    expected = {"START->1->2->4->5->END", "START->1->3->4->5->END"}
    result = g.get_paths()

    assert result == expected, f"Test failed! Got: {result}"
    print("Reachability test passed.")

# Run it:
test_reachability()

AssertionError: Test failed! Got: set()

In [3]:
def test_constraint_basic():
    g = CGraph()
    n1, n2, n3, n4, n5, n6 = [CGNode(i) for i in range(1, 7)]
    for n in [n1, n2, n3, n4, n5, n6]:
        g.add_node(n)

    g.add_edge(n1, n2, CGEdge.ADDR)
    g.add_edge(n3, n4, CGEdge.ADDR)
    g.add_edge(n2, n4, CGEdge.STORE)
    g.add_edge(n4, n5, CGEdge.COPY)
    g.add_edge(n5, n6, CGEdge.LOAD)

    g.solve_worklist()

    expected = {
        2: {1},
        3: {1},
        4: {3},
        5: {3},
        6: {1},
    }

    for node_id, pts in expected.items():
        actual = g.get_pts(node_id)
        assert actual == pts, f"Failed at node {node_id}: Expected {pts}, got {actual}"
    print("Basic constraint test passed.")

# Run it:
test_constraint_basic()

AssertionError: Failed at node 2: Expected {1}, got set()

In [4]:
def test_constraint_advanced():
    g = CGraph()
    nodes = {i: CGNode(i) for i in range(1, 9)}
    aux_nodes = {i: CGNode(i) for i in [11, 12, 13, 14]}
    all_nodes = {**nodes, **aux_nodes}

    for node in all_nodes.values():
        g.add_node(node)

    g.add_edge(aux_nodes[11], nodes[1], CGEdge.ADDR)
    g.add_edge(aux_nodes[12], nodes[2], CGEdge.ADDR)
    g.add_edge(aux_nodes[13], nodes[3], CGEdge.ADDR)
    g.add_edge(aux_nodes[14], nodes[4], CGEdge.ADDR)

    g.add_edge(nodes[1], nodes[3], CGEdge.STORE)
    g.add_edge(nodes[3], nodes[5], CGEdge.COPY)
    g.add_edge(nodes[5], nodes[7], CGEdge.LOAD)

    g.add_edge(nodes[2], nodes[4], CGEdge.STORE)
    g.add_edge(nodes[4], nodes[6], CGEdge.COPY)
    g.add_edge(nodes[6], nodes[8], CGEdge.LOAD)

    g.add_edge(nodes[8], nodes[5], CGEdge.STORE)
    g.add_edge(nodes[7], nodes[6], CGEdge.STORE)

    g.solve_worklist()

    expected = {
        1: {11},
        2: {12},
        3: {13},
        4: {14},
        5: {13},
        6: {14},
        7: {11, 12},
        8: {11, 12},
    }

    for node_id, pts in expected.items():
        actual = g.get_pts(node_id)
        assert actual == pts, f"Failed at node {node_id}: Expected {pts}, got {actual}"
    print("Advanced constraint test passed.")

# Run it:
test_constraint_advanced()

AssertionError: Failed at node 1: Expected {11}, got set()

## **Congratulations!**

Now, you've passed all tests—really great job! 🎉  
Your implementation is complete and working as expected.  
Keep up the excellent work and continue building your skills as a developer!
```